In [1]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

sys.path.append('../')
#from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *
from get_address_points import *

In [2]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [3]:
geo_key = config['keys']['geosupport_key']
geo_ip = config['keys']['geosupport_ip']

In [4]:
params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
                                 ';Database=' + structuresdb +
                                 ';Trusted_Connection=Yes;')
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection = engine.connect()

In [32]:
sql = 'select top 5 * from structuresdb.dbo.tbl_parks_structures'

In [33]:
b = pd.read_sql(con=engine, sql=sql)

In [37]:
structs = b[~pd.isnull(b['bin'])]

In [40]:
bins = list(structs.bin.values)

In [41]:
# bins = list(b[b['bin'].notnull()]['bin'])

In [42]:
# bins

In [9]:
aps, failed_bins = get_address_point(bins, geom_col = 'the_geom')

{'address_id': '5168623', 'the_geom': {'type': 'Point', 'coordinates': [-73.95761259499707, 40.728732897175995]}, 'bin': '3400033', 'h_no': '81', 'hyphen_typ': 'N', 'side_of_st': '1', 'borocode': '3', 'zipcode': '11222', 'created': '2015-12-15T00:00:00.000Z', 'st_name': 'FRANKLIN', 'physicalid': '64739', 'post_type': 'ST', 'full_stree': 'FRANKLIN ST'}
{'address_id': '3217153', 'the_geom': {'type': 'Point', 'coordinates': [-73.9999595066026, 40.59789328914727]}, 'bin': '3170079', 'h_no': '2134', 'hyphen_typ': 'N', 'side_of_st': '2', 'borocode': '3', 'zipcode': '11214', 'created': '2009-02-13T00:00:00.000Z', 'st_name': 'CROPSEY', 'physicalid': '42457', 'post_type': 'AVE', 'full_stree': 'CROPSEY AVE'}
{'address_id': '3217154', 'the_geom': {'type': 'Point', 'coordinates': [-74.001429101176, 40.59507913458522]}, 'bin': '3340691', 'h_no': '9000', 'hyphen_typ': 'N', 'side_of_st': '2', 'borocode': '3', 'zipcode': '11214', 'created': '2009-02-13T00:00:00.000Z', 'modified': '2018-07-26T00:00:00.

In [104]:
# type(aps)

In [106]:
aps

,address_id,the_geom,bin,h_no,hyphen_typ,side_of_st,borocode,zipcode,created,st_name,physicalid,post_type,full_stree,modified
0,5168623,POINT (inf inf),3400033,81,N,1,3,11222,2015-12-15T00:00:00.000Z,FRANKLIN,64739,ST,FRANKLIN ST,NaN
1,3217153,POINT (984261.245 157105.052),3170079,2134,N,2,3,11214,2009-02-13T00:00:00.000Z,CROPSEY,42457,AVE,CROPSEY AVE,NaN
2,3217154,POINT (983853.114 156079.787),3340691,9000,N,2,3,11214,2009-02-13T00:00:00.000Z,BAY,51350,PKWY,BAY PKWY,2018-07-26T00:00:00.000Z


In [12]:
failed_bins

[3423964.0]

In [16]:
aps.bin.values

array(['3400033', '3170079', '3340691'], dtype=object)

In [15]:
grc_err = ['01/F', '20', '21', '22', '23']
out_keys = [
    'AddressRangeList', 'out_bbl', 'out_TPAD_bin', 'out_TPAD_bin_status',
    'out_TPAD_conflict_flag', 'out_error_message', 'out_grc',
    'out_sanborn_boro', 'in_bin', 'out_boro_name1']

In [102]:
# for abin in bins
# bn = funcbn(bn = bins[2], out_keys = out_keys, grc_err = grc_err, api_key = geo_key, ip = geo_ip)

In [103]:
# structs['bin']

In [108]:
funcBN_outputs = structs.apply(lambda row: funcbn(row['bin'],
              out_keys=out_keys,
              grc_err=grc_err,
              api_key=geo_key,
              ip=geo_ip),axis =1)

10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3400033.0&key=flePZUDkE6sozZgz
10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3170079.0&key=flePZUDkE6sozZgz
10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3340691.0&key=flePZUDkE6sozZgz
10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3423964.0&key=flePZUDkE6sozZgz


In [109]:
t = flat_list(funcBN_outputs.values)
t

[{'b7sc': '34203001',
  'bin': '3400033',
  'high_address_number': '              81',
  'low_address_number': '              81',
  'street_name': 'FRANKLIN STREET                 ',
  'tpad_bin_status': 'N/A',
  'type': ' ',
  'type_meaning': 'Ordinary Address Range',
  'out_bbl': '3025650014',
  'out_TPAD_bin': 'N/A',
  'out_TPAD_bin_status': 'N/A',
  'out_TPAD_conflict_flag': 'N/A',
  'out_error_message': '                                                                                ',
  'out_grc': '00',
  'out_sanborn_boro': '3',
  'in_bin': '3400033',
  'out_boro_name1': 'BROOKLYN '},
 {'b7sc': '33063001',
  'bin': '3170079',
  'high_address_number': '            2134',
  'low_address_number': '            2134',
  'street_name': 'CROPSEY AVENUE                  ',
  'tpad_bin_status': 'N/A',
  'type': ' ',
  'type_meaning': 'Ordinary Address Range',
  'out_bbl': '3064900024',
  'out_TPAD_bin': 'N/A',
  'out_TPAD_bin_status': 'N/A',
  'out_TPAD_conflict_flag': 'N/A',
  'out_err

In [110]:
strip_vals(t)

In [111]:
t

[{'b7sc': '34203001',
  'bin': '3400033',
  'high_address_number': '81',
  'low_address_number': '81',
  'street_name': 'FRANKLIN STREET',
  'tpad_bin_status': 'N/A',
  'type': '',
  'type_meaning': 'Ordinary Address Range',
  'out_bbl': '3025650014',
  'out_TPAD_bin': 'N/A',
  'out_TPAD_bin_status': 'N/A',
  'out_TPAD_conflict_flag': 'N/A',
  'out_error_message': '',
  'out_grc': '00',
  'out_sanborn_boro': '3',
  'in_bin': '3400033',
  'out_boro_name1': 'BROOKLYN'},
 {'b7sc': '33063001',
  'bin': '3170079',
  'high_address_number': '2134',
  'low_address_number': '2134',
  'street_name': 'CROPSEY AVENUE',
  'tpad_bin_status': 'N/A',
  'type': '',
  'type_meaning': 'Ordinary Address Range',
  'out_bbl': '3064900024',
  'out_TPAD_bin': 'N/A',
  'out_TPAD_bin_status': 'N/A',
  'out_TPAD_conflict_flag': 'N/A',
  'out_error_message': '',
  'out_grc': '00',
  'out_sanborn_boro': '3',
  'in_bin': '3170079',
  'out_boro_name1': 'BROOKLYN'},
 {'b7sc': '31733001',
  'bin': '3340691',
  'high_a

In [112]:
BN_out_df = pd.DataFrame(t)
BN_out_df

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin,out_boro_name1
0,34203001,3400033,81,81,FRANKLIN STREET,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033,BROOKLYN
1,33063001,3170079,2134,2134,CROPSEY AVENUE,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079,BROOKLYN
2,31733001,3340691,9000,8922,BAY PARKWAY,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691,BROOKLYN
3,31049501,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964,BROOKLYN


In [82]:
# pd.DataFrame(strip_vals(flat_list(funcBN_outputs.values)))

In [17]:
bn

[{'b7sc': '31733001',
  'bin': '3340691',
  'high_address_number': '            9000',
  'low_address_number': '            8922',
  'street_name': 'BAY PARKWAY                     ',
  'tpad_bin_status': 'N/A',
  'type': ' ',
  'type_meaning': 'Ordinary Address Range',
  'out_bbl': '3064900024',
  'out_TPAD_bin': 'N/A',
  'out_TPAD_bin_status': 'N/A',
  'out_TPAD_conflict_flag': 'N/A',
  'out_error_message': '                                                                                ',
  'out_grc': '00',
  'out_sanborn_boro': '3',
  'in_bin': '3340691',
  'out_boro_name1': 'BROOKLYN '}]

In [113]:
# a = aps.iloc[0][['borocode', 'full_stree']]

In [114]:
# aps.values

In [115]:
func1N_outputs = aps.apply(lambda row: func1n(borough=row['borocode'],
                             streetname=row['full_stree'],
                             api_key=geo_key,
                             ip=geo_ip),axis =1)

In [116]:
list_1N = flat_list(func1N_outputs.values)

In [117]:
list_1N

[{'out_boro_name1': 'BROOKLYN ',
  'out_grc': '00',
  'out_stname1': 'FRANKLIN STREET                 '},
 {'out_boro_name1': 'BROOKLYN ',
  'out_grc': '00',
  'out_stname1': 'CROPSEY AVENUE                  '},
 {'out_boro_name1': 'BROOKLYN ',
  'out_grc': '00',
  'out_stname1': 'BAY PARKWAY                     '}]

In [118]:
strip_vals(list_1N)

In [119]:
list_1N

[{'out_boro_name1': 'BROOKLYN',
  'out_grc': '00',
  'out_stname1': 'FRANKLIN STREET'},
 {'out_boro_name1': 'BROOKLYN',
  'out_grc': '00',
  'out_stname1': 'CROPSEY AVENUE'},
 {'out_boro_name1': 'BROOKLYN', 'out_grc': '00', 'out_stname1': 'BAY PARKWAY'}]

In [120]:
func1N_out_df = pd.DataFrame(list_1N)

In [132]:
aps

,address_id,the_geom,bin,h_no,hyphen_typ,side_of_st,borocode,zipcode,created,st_name,physicalid,post_type,full_stree,modified
0,5168623,POINT (inf inf),3400033,81,N,1,3,11222,2015-12-15T00:00:00.000Z,FRANKLIN,64739,ST,FRANKLIN ST,NaN
1,3217153,POINT (984261.245 157105.052),3170079,2134,N,2,3,11214,2009-02-13T00:00:00.000Z,CROPSEY,42457,AVE,CROPSEY AVE,NaN
2,3217154,POINT (983853.114 156079.787),3340691,9000,N,2,3,11214,2009-02-13T00:00:00.000Z,BAY,51350,PKWY,BAY PKWY,2018-07-26T00:00:00.000Z


In [133]:
func1N_out_df

,out_boro_name1,out_grc,out_stname1,name_func_1n
0,BROOKLYN,00,FRANKLIN STREET,1N
1,BROOKLYN,00,CROPSEY AVENUE,1N
2,BROOKLYN,00,BAY PARKWAY,1N


In [123]:
BN_out_df['name_func_bn'] = 'BN'

In [122]:
func1N_out_df['name_func_1n'] = '1N'

In [137]:
aps_1N_df = aps.join(func1N_out_df)

In [141]:
aps_1N_df.columns.values

array(['address_id', 'the_geom', 'bin', 'h_no', 'hyphen_typ',
       'side_of_st', 'borocode', 'zipcode', 'created', 'st_name',
       'physicalid', 'post_type', 'full_stree', 'modified',
       'out_boro_name1', 'out_grc', 'out_stname1', 'name_func_1n'],
      dtype=object)

In [142]:
BN_out_df.columns.values

array(['b7sc', 'bin', 'high_address_number', 'low_address_number',
       'street_name', 'tpad_bin_status', 'type', 'type_meaning',
       'out_bbl', 'out_TPAD_bin', 'out_TPAD_bin_status',
       'out_TPAD_conflict_flag', 'out_error_message', 'out_grc',
       'out_sanborn_boro', 'in_bin', 'out_boro_name1', 'name_func_bn'],
      dtype=object)

In [143]:
merged_1N_and_BN = aps_1N_df.merge(BN_out_df, how = 'outer',on = 'bin')
merged_1N_and_BN.columns.values

array(['address_id', 'the_geom', 'bin', 'h_no', 'hyphen_typ',
       'side_of_st', 'borocode', 'zipcode', 'created', 'st_name',
       'physicalid', 'post_type', 'full_stree', 'modified',
       'out_boro_name1_x', 'out_grc_x', 'out_stname1', 'name_func_1n',
       'b7sc', 'high_address_number', 'low_address_number', 'street_name',
       'tpad_bin_status', 'type', 'type_meaning', 'out_bbl',
       'out_TPAD_bin', 'out_TPAD_bin_status', 'out_TPAD_conflict_flag',
       'out_error_message', 'out_grc_y', 'out_sanborn_boro', 'in_bin',
       'out_boro_name1_y', 'name_func_bn'], dtype=object)

In [145]:
merged_1N_and_BN.T

,0,1,2,3
address_id,5168623,3217153,3217154,NaN
the_geom,POINT (inf inf),POINT (984261.2452392586 157105.051635661),POINT (983853.1140136735 156079.7866210247),None
bin,3400033,3170079,3340691,3423964
h_no,81,2134,9000,NaN
hyphen_typ,N,N,N,NaN
side_of_st,1,2,2,NaN
borocode,3,3,3,NaN
zipcode,11222,11214,11214,NaN
created,2015-12-15T00:00:00.000Z,2009-02-13T00:00:00.000Z,2009-02-13T00:00:00.000Z,NaN
st_name,FRANKLIN,CROPSEY,BAY,NaN


In [20]:
func1n(borough = a['borocode'], streetname = a['full_stree'], api_key = geo_key, ip = geo_ip)

{'out_boro_name1': 'BROOKLYN ',
 'out_grc': '00',
 'out_stname1': 'FRANKLIN STREET                 '}

In [21]:
df = master_geosupport_func(bins)

10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3400033.0&key=flePZUDkE6sozZgz
10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3170079.0&key=flePZUDkE6sozZgz
10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3340691.0&key=flePZUDkE6sozZgz
10.155.226.59/geoservice/geoservice.svc/Function_BIN?BIN=3423964.0&key=flePZUDkE6sozZgz


In [22]:
df

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,...,out_san_sched,out_san_dist_section,out_san_recycle,out_san_reg,out_san_org_pick_up,out_usps_city_name,out_preferred_lgc,out_sos_ind,out_physical_id,out_ap_id
0,34203001,3400033,81,81,FRANKLIN STREET,None,None,Ordinary Address Range,3025650014,None,...,1A,301 / 011,EM,MWF,NaN,BROOKLYN,34203001,Address is on the left when facing from NOBLE ...,0064739,5168623
1,33063001,3170079,2134,2134,CROPSEY AVENUE,None,None,Ordinary Address Range,3064900024,None,...,5E,311 / 115,ES,WS,NaN,BROOKLYN,33063001,Address is on the right when facing from BAY ...,0042457,3217153
2,31733001,3340691,9000,8922,BAY PARKWAY,None,None,Ordinary Address Range,3064900024,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31049501,3423964,None,None,BELT PARKWAY EXIT 5 EASTBOUND,None,B,Non-Addressable Unnamed Building,3064900024,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
failed_bins

[3423964.0]

In [24]:
aps

,address_id,the_geom,bin,h_no,hyphen_typ,side_of_st,borocode,zipcode,created,st_name,physicalid,post_type,full_stree,modified
0,5168623,POINT (inf inf),3400033,81,N,1,3,11222,2015-12-15T00:00:00.000Z,FRANKLIN,64739,ST,FRANKLIN ST,NaN
1,3217153,POINT (984261.245 157105.052),3170079,2134,N,2,3,11214,2009-02-13T00:00:00.000Z,CROPSEY,42457,AVE,CROPSEY AVE,NaN
2,3217154,POINT (983853.114 156079.787),3340691,9000,N,2,3,11214,2009-02-13T00:00:00.000Z,BAY,51350,PKWY,BAY PKWY,2018-07-26T00:00:00.000Z
